In [73]:
import pickle
import numpy as np
import tensorflow as tf

image_size = 28
num_labels = 10

In [74]:
class NotMNIST:
    def __init__(self):
        class Train:
            def __init__(self):
                self.images = []
                self.labels = []
                self.batch_counter = 0

            @property
            def num_examples(self):
                return len(self.images)

            def next_batch(self, num):
                if self.batch_counter + num >= len(self.labels):
                    batch_images = self.images[self.batch_counter:]
                    batch_labels = self.labels[self.batch_counter:]
                    left = num - len(batch_labels)
                    self.batch_counter = left
                else:
                    batch_images = self.images[self.batch_counter:self.batch_counter + num]
                    batch_labels = self.labels[self.batch_counter:self.batch_counter + num]
                    self.batch_counter += num
                return batch_images, batch_labels

        class Test:
            def __init__(self):
                self.images = []
                self.labels = []

        self.train = Train()
        self.test = Test()
            
        self.load_as(self.format_of((-1, image_size * image_size)))
        
    def format_of(self, shape):
        return lambda dataset, labels: (
                dataset.reshape(shape).astype(np.float32),
                (np.arange(num_labels) == labels[:, None]).astype(np.float32)
            )
        
    def load_as(self, reformat):

        pickle_file = '../data/notMNIST.pickle'

        with open(pickle_file, 'rb') as f:
            save = pickle.load(f)
            train_dataset = save['train_dataset']
            train_labels = save['train_labels']
            valid_dataset = save['valid_dataset']
            valid_labels = save['valid_labels']
            test_dataset = save['test_dataset']
            test_labels = save['test_labels']
            del save  # 删除内存文件，等待gc回收释放内存
        
        train_dataset, train_labels = reformat(train_dataset, train_labels)
        valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
        test_dataset, test_labels = reformat(test_dataset, test_labels)
        print('Training set', train_dataset.shape, train_labels.shape)
        print('Validation set', valid_dataset.shape, valid_labels.shape)
        print('Test set', test_dataset.shape, test_labels.shape)
        self.train.images = train_dataset
        self.train.labels = train_labels
        self.test.images = test_dataset
        self.test.labels = test_labels  
        
    def reshape_like_image(self, channels):
        self.load_as(self.format_of((-1, image_size, image_size, channels)))      

not_mnist = NotMNIST()

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [76]:
num_channels = 1 # 灰度图

not_mnist.reshape_like_image(num_channels)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [79]:
# 训练参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 25  # 迭代轮数
batch_size = 50  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, image_size, image_size, num_channels])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, num_labels])  # 识别A-J的数据，一共10个类别

# CNN参数
patch_size = 5
depth = 16
num_hidden = 64

# Variables.
layer1_weights =
layer1_biases =

layer2_weights =
layer2_biases =

layer3_weights =
layer3_biases =

layer4_weights =
layer4_biases =

# Model.
def model(data):
    conv = 
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = 
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [-1, shape[1] * shape[2] * shape[3]])
    hidden = 
    return 

# 构造模型
pred = tf.nn.softmax(model(x))  # Softmax

# 损失函数：交叉墒
loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

def accuracy(pred, y, test_data):
    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy.eval({x: test_data.images, y: test_data.labels})

In [78]:
# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_loss = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, loss], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_loss += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "loss:", "{:.9f}".format(avg_loss),
                  "accuracy:", accuracy(pred, y, not_mnist.test))

    print("Optimization Finished!")
    print("Accuracy:", accuracy(pred, y, not_mnist.test))
    saver.save(sess, 'my-model')

Epoch: 0001 loss: 0.669210697 accuracy: 0.9112
Epoch: 0002 loss: 0.488070094 accuracy: 0.9267
Epoch: 0003 loss: 0.439474562 accuracy: 0.9355
Epoch: 0004 loss: 0.412151151 accuracy: 0.9396
Epoch: 0005 loss: 0.392998637 accuracy: 0.9431
Epoch: 0006 loss: 0.378263530 accuracy: 0.9441
Epoch: 0007 loss: 0.366172060 accuracy: 0.9449
Epoch: 0008 loss: 0.355846938 accuracy: 0.9468
Epoch: 0009 loss: 0.346964184 accuracy: 0.9471
Epoch: 0010 loss: 0.339002778 accuracy: 0.9476
Epoch: 0011 loss: 0.331863769 accuracy: 0.9478
Epoch: 0012 loss: 0.325323714 accuracy: 0.9497
Epoch: 0013 loss: 0.319461431 accuracy: 0.9502
Epoch: 0014 loss: 0.314051330 accuracy: 0.95
Epoch: 0015 loss: 0.309165841 accuracy: 0.9513
Epoch: 0016 loss: 0.304520438 accuracy: 0.9514
Epoch: 0017 loss: 0.300160999 accuracy: 0.9521
Epoch: 0018 loss: 0.296189671 accuracy: 0.9531
Epoch: 0019 loss: 0.292437611 accuracy: 0.9536
Epoch: 0020 loss: 0.288811665 accuracy: 0.9539
Epoch: 0021 loss: 0.285411709 accuracy: 0.9545
Epoch: 0022 los